In [4]:
import numpy as np
import pandas as pd

import tensorflow as tf
import keras
from keras.layers import *

In [8]:

#### Copy and paste this code fragment wherever you want to load the data

import dataloader

sparseAdWeights,adIds = dataloader.loadAdWeightsAndIds()
adIdsRev,convertToAdInx = dataloader.getAdIdConverterFunction(adIds)

eventsDf = pd.read_csv('filtered_events.csv')

uniqUser = np.unique(eventsDf.uuid)

In [6]:
eventsDf.head(10)

,display_id,uuid,ad_id,clicked,ad_inx,user_inx
0,10,b09a0e92aa4d17,241662,0,240230,0
1,10,b09a0e92aa4d17,332908,1,330782,0
2,30,53ca43b326638b,139944,1,139219,1
3,30,53ca43b326638b,166549,0,165588,1
4,30,53ca43b326638b,181548,0,180484,1
5,30,53ca43b326638b,250079,0,248581,1
6,30,53ca43b326638b,269017,0,267428,1
7,79,9d376148017289,79784,0,79292,2
8,79,9d376148017289,88711,0,88172,2
9,79,9d376148017289,92113,1,91552,2


## Probit model

In [10]:


def probit_activation(x):
    return tf.distributions.Normal(loc=0., scale=1.).cdf(x)


userInxInput = Input(shape=(1,))
adInxInput = Input(shape=(1,))

adWeightLayer = Embedding(
    sparseAdWeights.shape[0],
    sparseAdWeights.shape[1],
    input_length=1,
    trainable=False,
    weights=[sparseAdWeights.toarray()]
)(adInxInput)

userWeightLayer = Embedding(uniqUser.shape[0],sparseAdWeights.shape[1],input_length=1)(userInxInput)

dotLayer = Dot(-1)([adWeightLayer,userWeightLayer])

flat_ = Flatten()(dotLayer)

activationLayer = Activation(probit_activation)(flat_)

model = keras.models.Model(inputs=(userInxInput,adInxInput),outputs=(activationLayer))

model.compile(loss='mse', optimizer='adam') ## Maybe another optimizer?

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 96)        53719968    input_6[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1, 96)        9205824     input_5[0][0]                    
__________________________________________________________________________________________________
dot_1 (Dot

In [12]:
## TO DO: refine fit() params
model.fit(
    [eventsDf.user_inx,eventsDf.ad_inx],
    eventsDf.clicked,
    epochs = 10
)

Epoch 1/10
   4768/2405074 [..............................] - ETA: 1:40:24 - loss: 0.2505

KeyboardInterrupt: 

In [ ]:
## TODO: save model weights so they can be used later